# Modeling the Time-Dependent Pre-activation Signal Using Gaussian Processes

In [2]:
import numpy as np
import pandas as pd 
import bokeh.io 
import bokeh.plotting 
import pystan 
bokeh.io.output_notebook()

Loading BokehJS ...

In this notebook, I perform a cursory exploration of using Gaussian Process modeling to describe how the pre-activation intensity is dependent on time. The approach presented in this notebook is designed to be a section of a larger model that will consider all facets of the data as a single entity. Thus, this notebook serves as a proof-of-principle for pursuing this approach at a larger scale. 

In this notebook, I will very briefly summarize what a Gaussian process is and why it is applicable to this problem. I will then build a statistical model using the Stan probabilistic programming language to sample the parameter space for the GP hyperparameters. After defining the model,  I perform a series of checks to ensure the model works as advertised before applying it to the real-life data provided by Abrar. 

## WTF is a Gaussian Process?

**TL;DR - A Gaussian Process GP is a convenient way model the relationship between points without specifying a parametric relationship between them.**

In science, we often make experimental measurements of set of variates $x$ and covariates $y$. In many cases, we can perform a regression to identify the statistical relationship between $x$ and $y$, and this often involves estimating parameter values, such as the slope $\alpha$ and intercept $\beta$ in a linear regression. In such a case, we can estimate the posterior probability distribution of the parameters $\alpha, \beta$ given experimental measurements via Bayes' theorem, which states 

$$
P(\alpha,\beta,\vert\, x, y) = \frac{P(y, x\,\vert\,\alpha, \beta)P(\alpha,\beta)}{P(x, y)}. \tag{1}
$$

Here $P(\alpha,\beta\, \vert\, x, y)$ is the posterior distribution which defines a probability density over the parameters. The likelihood, $P(y, x\,\vert\, \alpha, \beta)$, captures the probability density of observing the data ($y, x$) given the parameters $\alpha$ and $\beta$, and the prior distribution $P(\alpha, \beta)$ captures all prior knowledge we have of the values of these parameters *without* considering the data. The denominator $P(x, y)$ is the likelihood marginalized over the parameters $\alpha$ and $\beta$, and is not particularly relevant to this discussion. 

While this is nice, it is not always the case that we have a parametric expectation of how the variats and covariats are related. A perfect example of this is precisely the topic of this notebook -- the spooky time-dependent pre-activation signal. We have no real *a priori* reason to believe that it should be linear, other than it kinda-sorta looks like it might be. Rather than performing parameteric inference on these data, we can try to describe it in a non-parametric way.


The formalism shown in Eq. 1 is true for any logical conjecture and is *not* restricted to computing probabilities over parameters. Rather, we could replace some of the terms in Eq. 1 to account for an unknown data-generating function $f$ as

$$
P(f\,\vert\, x, y) = \frac{P(x, y\,\vert\, f)P(f)}{P(x, y)}. \tag{2}
$$

It's a bit weird to think of probability densities over functions. We can instead think of $P(f\,\vert\, x, y)$ as a probability density over the values of a functions at a given finite number of points $\mathbf{X}$. Thus, with $N$ total observations (points), we can define a joint distribution over these values as $P(f(x_1), f(x_2), \dots, f(x_N))$ where $x_1, x_2, \dots, x_N \in \mathbf{X}$. For mathematical convenience (and courtesy of the central limit theorem), we can (and will) state that this joint distribution is a multi-variate Gaussian distribution with a mean function $m$ and a covariance function $k$. This defines a Gaussian Process (hereafter, GP).

## WTF is a "mean function"?
**TL;DR - Who cares? We can just set it to 0 (so long as are we are careful).**

Gaussian distributions are defined by location and scale parameters $\mu$, and $\sigma$, respectively. As we are considering a Gaussian distribution over possible values of an arbitrary function $f$, we must define the location and scale parameters as *functions* of the variate $x$. Thus, we can state that a given function $f$ evalated over a set of finite points $\mathbf{X}$ can be denoted as 

$$
f(\mathbf{X}) \sim \mathcal{GP}(m(\mathbf{X}), k(\mathbf{X})).
$$

If we have some prior knowledge of what this mean should be, we *could* prescribe this into our statistical model (Eq. 2). But, we don't have to. To remain completely non-parametric, we can just take the mean function evvaluated at a point to be 0, **so long as we center the measurements about 0**.  Since we wish to model the relationship between $x$ and $y$ about the mean of some unknown function, we can center and scale our data about 0 by 

$$ 
\mathbf{X}_{s} = \frac{\mathbf{X} - \bar{\mathbf{X}}}{\sigma_\mathbf{X}}, \tag{3}
$$

wehre $\bar{\mathbf{x}}$ is the arithmetic mean and $\sigma_\mathbf{X}$ is the variance. After we do our non-parametric inference, we can reverse this linear transformation to get back to the dimensions pertinent to our research question. 

## WTF is a "covariance function"?
**TL;DR - Covariance function is a kernel which describes how points are correlated. Lot's of choices, but you can usually use a squared exponential kernel (we will).**

A covariance function, or more simply termed a kernel, describes the different points are related to one another. In a multivariate world, we numerically identify how each pair of points are related to one another as a matrix. If we say that we have $N$ rows in our multivariate matrix $X$, then the covariance matrix is an $N\times N$ matrix with entries of 

$$
K_{i,j} = k(x_i, x_j). \tag{4}
$$
Because we are considering a multi-dimensional Gaussian distribution,  the covariance matrix $K(\mathbf{X}, \mathbf{X'})$ has some mathematical requirements (must be positive-definite for the curious nerds), meaning that we are restricted in our choices. There are a bunch of different kernels $k$ that can be used., with the most commonly used one being the squared-exponential kernel

$$
k(x_1, x_2) = \alpha^2 \exp \left(-\frac{(x_1 - x_2)^2}{\beta^2}\right), \tag{5}
$$

where $\alpha$ and $\beta$ are called **hyper-parameters** which we will have to infer later on. This is a very commonly used kernel and will be what we use here. 

## WTF is my prior knowledge of the possible function?
**TL;DR - For a GP, it has to be a multivariate normal (a "conjugate prior") defined by your kernel.**

Now that we've introduced a functional form for the kernel, we have some parameters that we need to care about. These hyperparameters, $\alpha,  \beta$ then inform our prior distribution over $f$  as  $P(f\,vert\, \alpha, \beta)$. Later on, we will put priors on the hyper-parameters themselves, creating a prior sandwich. 

For convvenience, we will say that the prior distribution over $f$ is a multivariate Gaussian distribution over a set of finite points where the variance is our covariance matrix $K(\mathbf{X}, \mathbf{X'})$ As the kernel describes how the points are related to one another, we can center our prior distribution again about 0, much as we did in defining the mean function, yielding 

$$
P(f\,\vert\, \alpha, \beta) = \text{Gaussian}(0, K(\mathbf{X}, \mathbf{X'})\,\vert\, \alpha, \beta). \tag{6}
$$

An immense convenience of choosing a multivariate normal distribution for the prior is that it is the conjugate prior for a multivariate normal likelihood (what we have!). This means that the posterior distribution *itself* is a multivariate normal distribution, making things much (much) easier for us to code and draw samples from. However, we do include one more parameter $\sigma$ which is a constant measurement error. As multiple measurements of the same variate can yield different vaulues, we can include this homoscedastic error into our kernel as 

$$
P(f(x)\,\vert\, x, y) \sim \mathcal{GP}(0, k(\mathbf{X}, \mathbf{X'} + \delta_{\mathbf{x}, \mathbf{x'}}\sigma)), \tag{7}
$$

where $\delta$ is a Kroneker delta function and is equal to unity only when the compared variates are identical. 

## Okay, what else?

I'm going to omit some of the mathy grunge for now, but Gaussian matrices have some neat tricks up their sleeves with regards to getting the conditional distribution from the joint distribution. The result is that we can solve for the mean value of the mean function (and therefore, the most likely value) through products of covariance matrices. These require tough (and computationally intensive) matrix inversions. In the code that will come in this document, we can trick our computer into computing the inverted matrices by doing something called a Cholesky decomposition.

I'm also leaving out some discussion of how we ultimately infer the values of the hyperparameters. This will be baked into the code to follow, but we can discuss this in depth at a later time. 

## The Data

Abrar sent me some processed data from two experiments where the preactivation signal was provided. We will load this data set and look only at the first experiment.

In [3]:
# Load the data. 
data = pd.read_csv('../processing/20200701_abrar_processed_munging/output/20200701_abrar_processed_data_tidy.csv')

# Restrict to the first experiment. 
experiment = data[data['experiment_id']==1]

# Set up a figure canvas. 
p = bokeh.plotting.figure(x_axis_label='time [min]', y_axis_label='preactivation intensity')
# Plot the points
p.circle(x='time_min', y='preact_intensity', source=experiment)
bokeh.io.show(p)

In [4]:
model_code = """

functions {
    // Function to generate posterior predictive samples
    vector gp_ppc_rng(
        real[] t_ppc,  // time points where to evaluate ppc
        vector y,  // intensity measurements
        real[] t_data,  // time points for intensities
        real alpha,  // marginal standard deviation
        real beta,  // time scale
        real sigma,  // measurement error
        real delta // extra value added for numerical stability
    ) {
    // Define variables for evaluation
    int N_data = rows(y);  // number of data points
    int N_ppc = size(t_ppc);  // number of time points on ppc
    vector[N_ppc] f_ppc;  //  posterior predictive samples
    {
        // Objective: 
        // Generate covariance matrix for the data
        matrix[N_data, N_data] K_exp = cov_exp_quad(t_data, alpha, beta);
        matrix[N_data, N_data] K = K_exp 
        + diag_matrix(rep_vector(square(sigma), N_data));
     
        // 1. Perform Cholesky decomposition K = L_K L_K'
        matrix[N_data, N_data] L_K = cholesky_decompose(K);
    

        // 2. Compute inv(L_K) y. Since L_K is a triangular matrix we can use
        // the mdivide_left_tri_low function
        vector[N_data] L_K_div_y_data = mdivide_left_tri_low(L_K, y);
        // With this result in hand we can now solve for alpha by computing
        // ⍺ = inv(L_k') inv(L_K) y. This is equivalent to ⍺ = inv(K) y

        // 3. Compute ⍺ = inv(L_k') inv(L_K) y. Since L_K' is a triangular 
        // matrix we can use the mdivide_right_tri_low function
        vector[N_data] K_div_y_data = mdivide_right_tri_low(
            L_K_div_y_data', L_K
        )';

        // Generate covariance matrix for both data an ppc
        matrix[N_data, N_ppc] k_t_data_t_ppc = cov_exp_quad(
            t_data, t_ppc, alpha, beta
        );
        // Evaluate the mean function of the Gaussian process, given by
  
        vector[N_ppc] f_mu = (k_t_data_t_ppc' * K_div_y_data);

        // Evaluate the variance function of the Gaussian process, given by
      
        // where K* is the covariance matrix of all the points to be evaluated.
        // Using the fact that inv(K) = inv(L_K L_K') this can be rewritten as
  
        // 1. Evaluate inv(L_K) K*
        matrix[N_data, N_ppc] v_pred = mdivide_left_tri_low(
            L_K, k_t_data_t_ppc
        );
        // 2. Evaluate  = K* - K*' inv(L_K L_K') K*
        matrix[N_ppc, N_ppc] cov_f_exp = cov_exp_quad(t_ppc, alpha, beta) ;
        matrix[N_ppc, N_ppc] cov_f_exp2 = cov_f_exp - v_pred' * v_pred;
        matrix[N_ppc, N_ppc] cov_f = cov_f_exp2 
                                      + diag_matrix(rep_vector(delta, N_ppc));

        // Generate random samples given the variance and covariance functions
        // for the ppc samples
        f_ppc = multi_normal_rng(f_mu, cov_f);
    }
    return f_ppc;
    }
}

data {
    // Data from intensity measurements
    int<lower=1> N;  // number of data points
    real t[N];  // time points where measurements were taken
    vector[N] y;  // intensity

    // Posterior Predictive Checks
    int<lower=1> N_predict;  // number of points where to evalute ppc
    real t_predict[N_predict];  // time points where to evaluate ppc
}

parameters {
    real<lower=0> beta;  // time scale
    real<lower=0> alpha;  // marginal standard deviation
    real<lower=0> sigma;  // measurement standard deviation
}

model {
    // Define covariance matrix k(t, t')
    matrix[N, N] cov_exp =  cov_exp_quad(t, alpha, beta);
    matrix[N, N] cov = cov_exp + diag_matrix(rep_vector(square(sigma), N));
    // Perform a Cholesky decomposition of the matrix, this means rewrite the
    // covariance matrix cov = L_cov L_cov'
    matrix[N, N] L_cov = cholesky_decompose(cov);
    
    // Sample data from a multinomial Gaussian with mean zero and rather than
    // covariance matrix, a Cholesky decomposed matrix
    y ~ multi_normal_cholesky(rep_vector(0, N), L_cov);
}

generated quantities {
    // Generate posterior predictive samples for the Gaussian process
    vector[N_predict] f_predict = gp_ppc_rng(
        t_predict, y, t, alpha, beta, sigma, 1e-10
    );
    // Generate posterior predictive samples for the observation process
    vector[N_predict] y_predict;
    for (n in 1:N_predict) {
        y_predict[n] = normal_rng(f_predict[n], sigma);
    }
}

"""
model = pystan.StanModel(model_code=model_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_ab86d49a0d716b59eac073ab84e91b35 NOW.


In [ ]:
N_predict = 500
t_predict = np.linspace(0, 200, N_predict)
t_predict_scaled = (t_predict - np.mean(t_predict)) / np.var(t_predict)
preact_scaled = (experiment['preact_intensity'].values - np.mean(experiment['preact_intensity'].values)) / np.var(experiment['preact_intensity'].values)
time_scaled = (experiment['time_min'].values - np.mean(experiment['time_min'].values)) / np.var(experiment['time_min'].values)  
data_dict = {'t_predict':t_predict_scaled, 'N_predict':N_predict, 'y':preact_scaled, 't':time_scaled, 'N':len(experiment)}
samples = model.sampling(data_dict)

In [ ]:
samples